These produce a final .json file that can be used as an input in chemiscope to produce the final figures of the manuscript. 
The file can be already found in the downloaded folder.
However, if you want to run the notebook yourself, a new chemiscope file will be generated, along with new pcovr figures. 

In [1]:
from data import DATA_3DCD, DATA_MP
import pickle
from IPython.display import clear_output
from sklearn.model_selection import learning_curve

PRODUCTION DATA PRESENT


In [2]:
%run ./modules.ipynb

sns.set(style="white", palette="muted", color_codes=True)
my_c = colors.ListedColormap(["mediumblue", "red"])
c = ["mediumblue", "red"]
sns.set_palette(sns.color_palette(c))

n_PC = 5  # number of principal components
mixing = 0.5  # beta parameter for pcvor

In [3]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                         Atoms:  1.0 KiB
                    Calculator:  1.0 KiB
                        Abinit:  1.0 KiB
                    Spacegroup:  1.0 KiB
                     DataFrame:  1.0 KiB
                          cmap:  1.0 KiB
                       Counter:  1.0 KiB
                   MaxNLocator:  1.0 KiB
                        Axes3D:  1.0 KiB
                        Line2D:  1.0 KiB


In [4]:
npzfile_3dcd = np.load(DATA_3DCD.soap, allow_pickle=True)
my_frames_3dcd = ase.io.read(DATA_3DCD.structures, index=":")
magic_3dcd = np.array(npzfile_3dcd["magic"], dtype=int).reshape(-1,1)
y_magic_3dcd = np.copy(magic_3dcd.reshape(-1, 1).ravel())

In [5]:
npzfile_mp = np.load(DATA_MP.soap, allow_pickle=True)
my_frames_mp = ase.io.read(DATA_MP.structures, index=":")
magic_mp = np.array(npzfile_mp["magic"], dtype=int).reshape(-1,1)
y_magic_mp = np.copy(magic_mp.reshape(-1, 1).ravel())

In [6]:
my_frames = np.array([*my_frames_3dcd, *my_frames_mp])
my_orig_frames = np.array([*my_frames_3dcd, *my_frames_mp])
del my_frames_mp, my_frames_3dcd

<ipython-input-6-89df66f8f642>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  my_frames = np.array([*my_frames_3dcd, *my_frames_mp])
<ipython-input-6-89df66f8f642>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  my_orig_frames = np.array([*my_frames_3dcd, *my_frames_mp])


In [7]:
y_magic = np.hstack((y_magic_3dcd, y_magic_mp))
del magic_mp, magic_3dcd, y_magic_3dcd, y_magic_mp

In [8]:
if os.path.exists('train_indices_all.npy'):
    i_train, i_test = np.load('train_indices_all.npy'), np.load('test_indices_all.npy')
    y_train, y_test = y_magic[i_train], y_magic[i_test]
else:
    print('generating')
    i_train, i_test, y_train, y_test = train_test_split(
        np.arange(X.shape[0]), y_magic, train_size=0.9
    )
    np.save('train_indices_all.npy', i_train)
    np.save('test_indices_all.npy', i_test)

In [9]:
r = 4.0
X_raw = np.load('soaps_{}.npy'.format(r))
x_scaler = pickle.load(open("x_scaler_blanked_{}.sav".format(r), 'rb'))
X = x_scaler.transform(X_raw)
X_train, X_test = X_raw[i_train], X_raw[i_test]
del X_raw
X_train = x_scaler.transform(X_train)
X_test = x_scaler.transform(X_test)
clf = pickle.load(open('random_forest_all_blanked_{}.sav'.format(r), 'rb'))

In [10]:
print("Accuracy on test set:" + str(clf.score(X_test, y_test.ravel())))

Accuracy on test set:0.8705523344522429


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


In [11]:
X_raw_3dcd = npzfile_3dcd["X_raw"]
X_raw_mp = npzfile_mp["X_raw"]
X_raw_tagged = np.vstack((X_raw_3dcd, X_raw_mp))
del X_raw_3dcd, X_raw_mp
x_tagged_scaler = StandardFlexibleScaler(column_wise=False).fit(X_raw_tagged)

In [12]:
X_tagged = x_tagged_scaler.transform(X_raw_tagged)
X_tagged_train = X_tagged[i_train]
X_tagged_test = X_tagged[i_test]
del X_raw_tagged, X_tagged

In [13]:
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import LinearSVC as SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier as SGDC
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

In [15]:
estimators = {"Random Forest": {"model": RandomForestClassifier(verbose=2, n_estimators=100, random_state=2, n_jobs=4),
                                "save_name": "random_forest_all"},
              "Decision Tree": {"model": DTC(random_state=2, max_features=80),
                                "save_name": "dtc_all"},
              "Linear SVM": {"model": SVC(verbose=2, random_state=2),
                             "save_name": "svc_all"},
              "Cross-Validated Logistic Regression": {"model": LogisticRegressionCV(verbose=2, cv=2, random_state=2, n_jobs=4),
                                                      "save_name": "cvlr_all"},
              "Stochastic Gradient Descent Classifier": {"model": SGDC(verbose=2, n_jobs=4, max_iter=100, random_state=2),
                                                         "save_name": "sgdc_all"},
              "QDA": {"model":  QDA(),
                      "save_name": "qda_all"},
              "Gaussian Naive Bayes": {"model": GNB(),
                      "save_name": "gnb"
                     },
              "MLP Classifier": {"model": MLP(verbose=2, random_state=2),
                      "save_name": "mlp"
                     }
             }

In [18]:
e = "MLP Classifier"
v = estimators[e]

In [19]:
save_name = v['save_name']
model = v['model']
model.fit(X_tagged_train, y_train)
pickle.dump(model, open(save_name+'.sav', "wb"))

Iteration 1, loss = 0.64023381
Iteration 2, loss = 0.61017421
Iteration 3, loss = 0.59590966
Iteration 4, loss = 0.58488087
Iteration 5, loss = 0.57609234
Iteration 6, loss = 0.56742792
Iteration 7, loss = 0.56001116
Iteration 8, loss = 0.55325943
Iteration 9, loss = 0.54682541
Iteration 10, loss = 0.53946351
Iteration 11, loss = 0.53449691
Iteration 12, loss = 0.52825930
Iteration 13, loss = 0.52241832
Iteration 14, loss = 0.51727602
Iteration 15, loss = 0.51246324
Iteration 16, loss = 0.50764468
Iteration 17, loss = 0.50300101
Iteration 18, loss = 0.49837609
Iteration 19, loss = 0.49455030
Iteration 20, loss = 0.49022708
Iteration 21, loss = 0.48652029
Iteration 22, loss = 0.48215757
Iteration 23, loss = 0.47883535
Iteration 24, loss = 0.47545474
Iteration 25, loss = 0.47167190
Iteration 26, loss = 0.46797394
Iteration 27, loss = 0.46499337
Iteration 28, loss = 0.46216161
Iteration 29, loss = 0.45833395
Iteration 30, loss = 0.45597464
Iteration 31, loss = 0.45209963
Iteration 32, los

/opt/miniconda3/envs/aiida/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [20]:
model = pickle.load(open(save_name+'.sav', "rb"))
estimators[e]['tagged_error'] = model.score(X_tagged_test, y_test)
print(estimators[e].get('blanked_error'), estimators[e].get('tagged_error'))

None 0.8007934086054318


In [21]:
model = v['model']
pickle.dump(model.fit(X_train, y_train), open(save_name+'_XX.sav', "wb"))

Iteration 1, loss = 0.66417706
Iteration 2, loss = 0.64942005
Iteration 3, loss = 0.64259572
Iteration 4, loss = 0.63855252
Iteration 5, loss = 0.63587182
Iteration 6, loss = 0.63308108
Iteration 7, loss = 0.63076169
Iteration 8, loss = 0.62860450
Iteration 9, loss = 0.62714462
Iteration 10, loss = 0.62522076
Iteration 11, loss = 0.62346145
Iteration 12, loss = 0.62221410
Iteration 13, loss = 0.62052358
Iteration 14, loss = 0.61892720
Iteration 15, loss = 0.61819800
Iteration 16, loss = 0.61676609
Iteration 17, loss = 0.61524512
Iteration 18, loss = 0.61419796
Iteration 19, loss = 0.61308426
Iteration 20, loss = 0.61185243
Iteration 21, loss = 0.61133653
Iteration 22, loss = 0.61003837
Iteration 23, loss = 0.60914118
Iteration 24, loss = 0.60780729
Iteration 25, loss = 0.60707316
Iteration 26, loss = 0.60626153
Iteration 27, loss = 0.60529669
Iteration 28, loss = 0.60474863
Iteration 29, loss = 0.60368951
Iteration 30, loss = 0.60286824
Iteration 31, loss = 0.60219986
Iteration 32, los

/opt/miniconda3/envs/aiida/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
class_table = ('\\begin{table}[htbp!]'+
'\n\\centering\n\\begin{tabular}{|m{2.0cm}|m{2.0cm}|m{10cm}|}'+
'\n\\hline\nClassifier & Test Set R$^2$ & Test Set R$^2$ &Classifier parameters\\\\'+
'\n\\hline\n & (Species-Invariant) & (Species-Tagged) &\\\\'+
'\n\\hline'+
'\n\\hline')

for e,v in estimators.items():
    save_name = v['save_name']
    if 'blanked_error' not in v:
        if not os.path.exists(save_name+'_XX.sav'):
            print(f"Fitting {e}")
            model = v['model']
            pickle.dump(model.fit(X_train, y_train), open(save_name+'_XX.sav', "wb"))
            print(save_name+'_XX.sav')
        model = pickle.load(open(save_name+'_XX.sav', "rb"))
        estimators[e]['blanked_error'] = model.score(X_test, y_test)
    if 'tagged_error' not in v:
#         if not os.path.exists(save_name+'.sav'):
#             print(f"Fitting {e}")
#             model = v['model']
#             pickle.dump(model.fit(X_tagged_train, y_train), open(save_name+'.sav', "wb"))
#             print(save_name+'.sav')
        if os.path.exists(save_name+'.sav'):
            model = pickle.load(open(save_name+'.sav', "rb"))
            estimators[e]['tagged_error'] = model.score(X_tagged_test, y_test)
            
    params = ', '.join([f'\\texttt{{{k.replace("_", " ")}}}: {v.replace("_", " ") if isinstance(v, str) else v}' for k, v in estimators[e]['model'].get_params().items()])
    class_table += f'\n{e} & {round(estimators[e].get("blanked_error", -1),3)} & {round(estimators[e].get("tagged_error", -1),3)} &\\{{{params}\\}}\\\\\n\\hline'
    
class_table += ('\\end{tabular}'+
'\n\\caption{Accuracy on test set achieved by different classifiers.}  '+
'\n\\label{table:classif}'
'\n\\end{table}')

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.4s finished


In [23]:
print(class_table)

\begin{table}[htbp!]
\centering
\begin{tabular}{|m{2.0cm}|m{2.0cm}|m{10cm}|}
\hline
Classifier & Test Set R$^2$ & Test Set R$^2$ &Classifier parameters\\
\hline
 & (Species-Invariant) & (Species-Tagged) &\\
\hline
\hline
Random Forest & 0.871 & 0.88 &\{\texttt{bootstrap}: True, \texttt{ccp alpha}: 0.0, \texttt{class weight}: None, \texttt{criterion}: gini, \texttt{max depth}: None, \texttt{max features}: sqrt, \texttt{max leaf nodes}: None, \texttt{max samples}: None, \texttt{min impurity decrease}: 0.0, \texttt{min samples leaf}: 1, \texttt{min samples split}: 2, \texttt{min weight fraction leaf}: 0.0, \texttt{n estimators}: 100, \texttt{n jobs}: 4, \texttt{oob score}: False, \texttt{random state}: 2, \texttt{verbose}: 2, \texttt{warm start}: False\}\\
\hline
Decision Tree & 0.587 & 0.805 &\{\texttt{ccp alpha}: 0.0, \texttt{class weight}: None, \texttt{criterion}: gini, \texttt{max depth}: None, \texttt{max features}: 80, \texttt{max leaf nodes}: None, \texttt{min impurity decrease}: 